In [21]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import cv2
# from evaluator import ModelEvaluator
from tqdm import tqdm
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis, parameter_count
from ptflops import get_model_complexity_info
import time

In [22]:
class ModelEvaluator:
    def __init__(self, model, device=None):
        """
        모델 평가 클래스 초기화
        Args:
            model (torch.nn.Module): 평가할 PyTorch 모델
            device (torch.device, optional): 사용할 디바이스 (CPU/GPU)
        """
        self.model = model
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def evaluate_model_size(self):
        """
        모델의 총 파라미터 수와 학습 가능한 파라미터 수를 출력
        """
        params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print(f"Total Parameters: {params:,}")
        print(f"Trainable Parameters: {trainable_params:,}")
        return params, trainable_params

    def evaluate_flops(self, input_size=(3, 128, 112)):
        """
        모델의 FLOPs(Floating Point Operations) 계산
        Args:
            input_size (tuple): 입력 텐서 크기 (채널, 높이, 너비)
        """
        self.model.eval()
        with torch.no_grad():
            macs, params = get_model_complexity_info(self.model, input_size, as_strings=False, verbose=False)
            flops = macs * 2  # FLOPs = MACs * 2
            print(f"FLOPs: {flops / 1e6:.2f} MFLOPs")
        return flops


    def evaluate_inference_speed(self, input_size=(3, 128, 128), iterations=100):
        """
        모델의 평균 추론 속도 측정
        Args:
            input_size (tuple): 입력 텐서 크기 (채널, 높이, 너비)
            iterations (int): 추론 반복 횟수
        """
        self.model.eval()
        input_tensor = torch.randn(1, *input_size).to(self.device)
        torch.cuda.synchronize()  # GPU 사용 시 동기화

        start_time = time.time()
        for _ in range(iterations):
            with torch.no_grad():
                self.model(input_tensor)
        torch.cuda.synchronize()  # GPU 사용 시 동기화
        end_time = time.time()

        avg_time_per_inference = (end_time - start_time) / iterations
        print(f"Average Inference Time: {avg_time_per_inference * 1000:.2f} ms")
        return avg_time_per_inference

    def summarize_model(self, input_size=(3, 128, 128)):
        """
        모델의 레이어별 출력 크기와 파라미터 요약
        Args:
            input_size (tuple): 입력 텐서 크기 (채널, 높이, 너비)
        """
        print("\n===== Model Summary =====")
        summary(self.model, input_size=input_size)

    def evaluate_all(self, input_size=(3, 128, 128), iterations=100):
        """
        모델의 모든 평가(파라미터 수, FLOPs, 실행 속도, 요약)를 수행
        Args:
            input_size (tuple): 입력 텐서 크기 (채널, 높이, 너비)
            iterations (int): 추론 반복 횟수
        """
        print("\n===== Model Evaluation =====")
        self.evaluate_model_size()
        self.evaluate_flops(input_size=input_size)
        # self.evaluate_inference_speed(input_size=input_size, iterations=iterations)
        self.summarize_model(input_size=input_size)


In [23]:
# Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [24]:
# 데이터셋 경로
data_dir = "data"

In [25]:
# 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [26]:
# 데이터셋 로드
dataset = ImageFolder(root=data_dir, transform=transform)
# dataset = ImageFolder(root=data_dir)

# 클래스 정보 출력
print(f"Classes: {dataset.classes}")

Classes: ['with_mask', 'without_mask']


In [27]:
# Train:Val:Test = 70:15:15 분할
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])


In [28]:
# DataLoader 생성
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [29]:
class MaskClassifier(nn.Module):
    def __init__(self):
        super(MaskClassifier, self).__init__()
        
        # Feature Extraction - 더 얕은 구조로 변경
        self.features = nn.Sequential(
            # First Block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Second Block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Third Block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
        )
        
        # Classifier - 더 단순한 구조로 변경
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [30]:
model = MaskClassifier().to(device)

In [31]:
criterion = nn.CrossEntropyLoss()
# Loss function 수정 - 클래스 불균형을 고려한 가중치 추가
# weights = torch.tensor([1.0, 1.0]).to(device)  # 필요시 클래스별 가중치 조정
# criterion = nn.CrossEntropyLoss(weight=weights)

# Learning rate와 optimizer 수정
learning_rate = 0.001  # 더 작은 learning rate 사용
num_epoch = 50  # epoch 수 감소

optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    weight_decay=0.0001  # 더 작은 weight decay
)

In [32]:
def train_model(model, train_loader, val_loader, epochs=10):
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch")
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Progress Bar에 현재 배치의 accuracy 표시
            batch_acc = 100. * correct / total
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{batch_acc:.2f}%'
            })
    
    # 최종 학습 결과 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    final_acc = 100. * correct / total
    print("\n=== Final Training Results ===")
    print(f"Final Validation Accuracy: {final_acc:.2f}%")

In [33]:
# 학습 실행
train_model(model, train_loader, val_loader, num_epoch)

Epoch 50/50: 100%|██████████| 72/72 [00:39<00:00,  1.84batch/s, loss=0.0219, acc=93.86%]



=== Final Training Results ===
Final Validation Accuracy: 94.67%


In [34]:
# 테스트 함수
def test_model(model, test_loader):
    """
    Args:
        model (torch.nn.Module): 평가할 모델
        test_loader (DataLoader): 테스트 데이터 로더
    """
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Testing", unit="batch")  # Progress Bar 추가
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

            # Progress Bar 상태 업데이트 (현재 배치의 예측 결과 일부 표시)
            progress_bar.set_postfix(batch_accuracy=(preds == labels).float().mean().item())

    print("\nTest Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))


In [35]:
# 테스트 실행
test_model(model, test_loader)

Testing: 100%|██████████| 16/16 [00:03<00:00,  4.79batch/s, batch_accuracy=1]    


Test Classification Report:
              precision    recall  f1-score   support

   with_mask       1.00      0.92      0.96       246
without_mask       0.92      1.00      0.96       243

    accuracy                           0.96       489
   macro avg       0.96      0.96      0.96       489
weighted avg       0.96      0.96      0.96       489



In [36]:
# 웹캠 테스트 함수
def webcam_test(model):
    model.eval()
    cap = cv2.VideoCapture(0)  # 웹캠 열기
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    print("Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            _, pred = torch.max(output, 1)

        label = dataset.classes[pred.item()]
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [37]:
# 웹캠 테스트 실행
webcam_test(model)

Press 'q' to quit.


In [38]:
# 모델 저장 코드
def save_model(model, path="mask_classifier.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [42]:
# 학습 후 모델 저장 및 평가
save_model(model, "mask_classifier.pth")

Model saved to mask_classifier.pth


In [40]:
# 모델 정의 및 학습 완료 후
evaluator = ModelEvaluator(model, device=device)
evaluator.evaluate_all(input_size=(3, 128, 128), iterations=100)


===== Model Evaluation =====
Total Parameters: 93,954
Trainable Parameters: 93,954
MaskClassifier(
  93.95 k, 100.000% Params, 169.77 MMac, 98.912% MACs, 
  (features): Sequential(
    93.7 k, 99.725% Params, 169.74 MMac, 98.893% MACs, 
    (0): Conv2d(896, 0.954% Params, 14.68 MMac, 8.553% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, 0.068% Params, 1.05 MMac, 0.611% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(0, 0.000% Params, 524.29 KMac, 0.305% MACs, inplace=True)
    (3): MaxPool2d(0, 0.000% Params, 524.29 KMac, 0.305% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(0, 0.000% Params, 0.0 Mac, 0.000% MACs, p=0.2, inplace=False)
    (5): Conv2d(18.5 k, 19.686% Params, 75.76 MMac, 44.139% MACs, 32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(128, 0.136% Params, 524.29 KMac, 0.305% MACs, 64, eps=1e-05, momentum=0.1, affine=Tr